## Exploring and predicting

Import the libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Load the data into data frame

In [2]:
real_train_data = pd.read_csv("./data/train.csv")
real_test_data = pd.read_csv("./data/test.csv")
real_train_data.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
95846,999977655955,"""\nI have discussed it, unlike most of those w...",0,0,0,0,0,0
95847,999982426659,"ps. Almost forgot, Paine don't reply back to t...",1,0,1,0,0,0
95848,999982764066,Mamoun Darkazanli\nFor some reason I am unable...,0,0,0,0,0,0
95849,999986890563,Salafi would be a better term. It is more poli...,0,0,0,0,0,0
95850,999988164717,making wikipedia a better and more inviting pl...,0,0,0,0,0,0


In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(real_train_data, test_size=0.2)
train_data.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
61215,637851437405,Hey idiot! What's this fucking and animal lang...,1,1,1,0,1,0
12355,127430748543,""" June 2010 (UTC)\n\n I would suggest, quite s...",0,0,0,0,0,0
18734,194109274271,There does not seem to be a cohesive entity ca...,0,0,0,0,0,0
55989,583947631711,Please check the defintion of Category:Concept...,0,0,0,0,0,0
60337,628918345294,"""\n\nQuestion - does Wikipedia have articles o...",0,0,0,0,0,0


In [4]:
train_labels = train_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
train_features = train_data['comment_text']

Building pipeline for training using Multinomial Naive Bayes algorythm.

In [5]:
# create pipeline
from sklearn.multiclass import OneVsRestClassifier
clf = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words='english')),
    ('classifier', OneVsRestClassifier(MultinomialNB())),
])

In [6]:
clf.fit(train_features.values, train_labels.values)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...assifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          n_jobs=1))])

Test messages

In [7]:
test_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
1304,13628510767,"Hi Elen, apart from chewing your ear for the f...",0,0,0,0,0,0
69433,724406376236,See claim three here: http://onlinelibrary.wil...,0,0,0,0,0,0
4400,45872162686,complete bullshit - your past is catching up w...,1,0,1,0,0,0
64751,674654617104,"I'm away from home now... Basically, I returne...",0,0,0,0,0,0
52060,543151192790,Think Breast's\nThink Breast's\nThink Breast's...,0,0,0,0,0,0


In [8]:
columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] # do not remove
predicted = list(clf.predict(test_data['comment_text'].values))
real_labels = list(test_data[columns].values)

# predicted_df = test_data[['id', 'comment_text']]

In [9]:
predicted_labels_df = pd.DataFrame(predicted, columns=columns)

In [10]:
predicted_labels_df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [11]:
from sklearn.metrics import accuracy_score

In [12]:
accuracy_toxic = accuracy_score(test_data['toxic'].values, predicted_labels_df['toxic'].values)
accuracy_severe_toxic = accuracy_score(test_data['severe_toxic'].values, predicted_labels_df['severe_toxic'].values)
accuracy_obscene = accuracy_score(test_data['obscene'].values, predicted_labels_df['obscene'].values)
accuracy_threat = accuracy_score(test_data['threat'].values, predicted_labels_df['threat'].values)
accuracy_insult = accuracy_score(test_data['insult'].values, predicted_labels_df['insult'].values)
accuracy_identity_hate = accuracy_score(test_data['identity_hate'].values, predicted_labels_df['identity_hate'].values)

In [13]:
print("Accuracy of toxic comment prediction is ", accuracy_toxic)
print("Accuracy of severe toxic comment prediction is ", accuracy_severe_toxic)
print("Accuracy of obscene comment prediction is ", accuracy_obscene)
print("Accuracy of threat comment prediction is ", accuracy_threat)
print("Accuracy of insult comment prediction is ", accuracy_insult)
print("Accuracy of comment with identity hate prediction is ", accuracy_identity_hate)

Accuracy of toxic comment prediction is  0.917583850608
Accuracy of severe toxic comment prediction is  0.990037035105
Accuracy of obscene comment prediction is  0.950550310365
Accuracy of threat comment prediction is  0.997339731887
Accuracy of insult comment prediction is  0.951228417923
Accuracy of comment with identity hate prediction is  0.99201919566


It seems that the accuracy is pretty high. But the reason here is that the most of comments have zeros. So, the model is pretty good in predicting that the comment is unlikely to fall in one of the categories.

What can be done to measure the real accuracy?
1. Seems that dropping off the neutral comments will show the real accuracy. Probably, I have to try to cut off comments with all zeros (neutral comments).

In [14]:
# test = test[(test['id'] != 1) | (test['num'] != 1)]
test_offensive_df = test_data[(test_data['toxic'] != 0) 
                              | (test_data['severe_toxic'] != 0) 
                              | (test_data['obscene'] != 0) 
                              | (test_data['threat'] != 0) 
                              | (test_data['insult'] != 0) 
                              | (test_data['identity_hate'] != 0)]

Let's make sure that all of the messages have at least one '1'.

In [15]:
test_offensive_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
4400,45872162686,complete bullshit - your past is catching up w...,1,0,1,0,0,0
22629,233569787589,""" name was Willie, and he said he was standing...",0,0,0,0,0,1
26870,278354806104,sup bitch i'm back. yeah its 420 MOONSHINE bA...,1,0,1,0,1,0
40220,420211466315,Vandalism of my user page \n\nPlease do not va...,1,0,1,0,1,0
55082,574996253054,November 2012 (UTC) | decline=You moron! Why d...,1,0,0,0,1,0


Predict the values for the new dataframe.

In [16]:
predicted_toxicity = list(clf.predict(test_offensive_df['comment_text'].values))
real_labels_toxicity = list(test_offensive_df[columns].values)

# predicted_df = test_data[['id', 'comment_text']]
predicted_labels_df = pd.DataFrame(predicted_toxicity, columns=columns)
predicted_labels_df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,1,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


Calculate new predictions for toxic comments. But I already anticipate the result...

In [17]:
accuracy_toxic_n = accuracy_score(test_offensive_df['toxic'].values, predicted_labels_df['toxic'].values)
accuracy_severe_toxic_n = accuracy_score(test_offensive_df['severe_toxic'].values, predicted_labels_df['severe_toxic'].values)
accuracy_obscene_n = accuracy_score(test_offensive_df['obscene'].values, predicted_labels_df['obscene'].values)
accuracy_threat_n = accuracy_score(test_offensive_df['threat'].values, predicted_labels_df['threat'].values)
accuracy_insult_n = accuracy_score(test_offensive_df['insult'].values, predicted_labels_df['insult'].values)
accuracy_identity_hate_n = accuracy_score(test_offensive_df['identity_hate'].values, predicted_labels_df['identity_hate'].values)

In [18]:
print("Accuracy of toxic comment prediction is ", accuracy_toxic_n)
print("Accuracy of severe toxic comment prediction is ", accuracy_severe_toxic_n)
print("Accuracy of obscene comment prediction is ", accuracy_obscene_n)
print("Accuracy of threat comment prediction is ", accuracy_threat_n)
print("Accuracy of insult comment prediction is ", accuracy_insult_n)
print("Accuracy of comment with identity hate prediction is ", accuracy_identity_hate_n)

Accuracy of toxic comment prediction is  0.204831404127
Accuracy of severe toxic comment prediction is  0.903875188727
Accuracy of obscene comment prediction is  0.522898842476
Accuracy of threat comment prediction is  0.974333165576
Accuracy of insult comment prediction is  0.529441368898
Accuracy of comment with identity hate prediction is  0.922999496729


Ok, so the most common type of toxic comments - toxic comment - is the most badly predicted. Again, there is obviously a correlation (negative) between the *toxicity* of a comment and the eventual accuracy of the model.

### Try changing vectorizer

In [19]:
clf = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('classifier', OneVsRestClassifier(MultinomialNB())),
])

clf.fit(train_features.values, train_labels.values)
columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] # do not remove
predicted = list(clf.predict(test_data['comment_text'].values))
real_labels = list(test_data[columns].values)
#predicted_labels_df = pd.DataFrame(predicted, columns=columns)

test_offensive_df = test_data[(test_data['toxic'] != 0) 
                              | (test_data['severe_toxic'] != 0) 
                              | (test_data['obscene'] != 0) 
                              | (test_data['threat'] != 0) 
                              | (test_data['insult'] != 0) 
                              | (test_data['identity_hate'] != 0)]

predicted_toxicity = list(clf.predict(test_offensive_df['comment_text'].values))
real_labels_toxicity = list(test_offensive_df[columns].values)

# predicted_df = test_data[['id', 'comment_text']]
predicted_labels_df = pd.DataFrame(predicted_toxicity, columns=columns)
predicted_labels_df.head()



,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,1,0,1,1,1,0
3,1,0,0,0,0,0
4,0,0,0,0,0,0


In [20]:
accuracy_toxic_cv = accuracy_score(test_offensive_df['toxic'].values, predicted_labels_df['toxic'].values)
accuracy_severe_toxic_cv = accuracy_score(test_offensive_df['severe_toxic'].values, predicted_labels_df['severe_toxic'].values)
accuracy_obscene_cv = accuracy_score(test_offensive_df['obscene'].values, predicted_labels_df['obscene'].values)
accuracy_threat_cv = accuracy_score(test_offensive_df['threat'].values, predicted_labels_df['threat'].values)
accuracy_insult_cv = accuracy_score(test_offensive_df['insult'].values, predicted_labels_df['insult'].values)
accuracy_identity_hate_cv = accuracy_score(test_offensive_df['identity_hate'].values, predicted_labels_df['identity_hate'].values)

In [21]:
print("Accuracy of toxic comment prediction is ", accuracy_toxic_cv)
print("Accuracy of severe toxic comment prediction is ", accuracy_severe_toxic_cv)
print("Accuracy of obscene comment prediction is ", accuracy_obscene_cv)
print("Accuracy of threat comment prediction is ", accuracy_threat_cv)
print("Accuracy of insult comment prediction is ", accuracy_insult_cv)
print("Accuracy of comment with identity hate prediction is ", accuracy_identity_hate_cv)

Accuracy of toxic comment prediction is  0.588827377957
Accuracy of severe toxic comment prediction is  0.89028686462
Accuracy of obscene comment prediction is  0.733266230498
Accuracy of threat comment prediction is  0.970306995471
Accuracy of insult comment prediction is  0.688475088072
Accuracy of comment with identity hate prediction is  0.909411172622


Seems that CountVectorizer makes better job. Let's also change classifier to Random Forests.

In [22]:
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('classifier', OneVsRestClassifier(RandomForestClassifier())),
])

In [23]:
clf.fit(train_features.values, train_labels.values)
columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] # do not remove
predicted = list(clf.predict(test_data['comment_text'].values))
real_labels = list(test_data[columns].values)

test_offensive_df = test_data[(test_data['toxic'] != 0) 
                              | (test_data['severe_toxic'] != 0) 
                              | (test_data['obscene'] != 0) 
                              | (test_data['threat'] != 0) 
                              | (test_data['insult'] != 0) 
                              | (test_data['identity_hate'] != 0)]

predicted_toxicity = list(clf.predict(test_offensive_df['comment_text'].values))
real_labels_toxicity = list(test_offensive_df[columns].values)

predicted_labels_df = pd.DataFrame(predicted_toxicity, columns=columns)


In [27]:
accuracy_toxic_rf = accuracy_score(test_offensive_df['toxic'].values, predicted_labels_df['toxic'].values)
accuracy_severe_toxic_rf = accuracy_score(test_offensive_df['severe_toxic'].values, predicted_labels_df['severe_toxic'].values)
accuracy_obscene_rf = accuracy_score(test_offensive_df['obscene'].values, predicted_labels_df['obscene'].values)
accuracy_threat_rf = accuracy_score(test_offensive_df['threat'].values, predicted_labels_df['threat'].values)
accuracy_insult_rf = accuracy_score(test_offensive_df['insult'].values, predicted_labels_df['insult'].values)
accuracy_identity_hate_rf = accuracy_score(test_offensive_df['identity_hate'].values, predicted_labels_df['identity_hate'].values)

In [28]:
print("Accuracy of toxic comment prediction is ", accuracy_toxic_rf)
print("Accuracy of severe toxic comment prediction is ", accuracy_severe_toxic_rf)
print("Accuracy of obscene comment prediction is ", accuracy_obscene_rf)
print("Accuracy of threat comment prediction is ", accuracy_threat_rf)
print("Accuracy of insult comment prediction is ", accuracy_insult_rf)
print("Accuracy of comment with identity hate prediction is ", accuracy_identity_hate_rf)

Accuracy of toxic comment prediction is  0.573225968797
Accuracy of severe toxic comment prediction is  0.90437845999
Accuracy of obscene comment prediction is  0.78208354303
Accuracy of threat comment prediction is  0.975339708103
Accuracy of insult comment prediction is  0.676396577755
Accuracy of comment with identity hate prediction is  0.924006039255


It appears that RandomForestsClassifier doesn't make any difference compare to Multinomial Naive Bayes classifier. Since it takes longer to train on random forests, Naive Bayes is a better choice.